# Part 2: Launch an OpenGrid Node On Heroku

OpenGrid (or "Grid") is the platform library supporting the deployment of libraries for privacy-preserving artificial intelligence. In this tutorial, you'll learn how to deploy a grid node onto Heroku and then interact with it using PySyft.

_WARNING: Grid nodes publish datasets online and are for EXPERIMENTAL use only. Deploy nodes at your own risk. Do not use OpenGrid with any data/models you wish to keep private._

### Step 1: Setup an account on Heroku.

Go to http://heroku.com and setup a username and password.

### Step 2: Verify your Heroku Account

So while this tutorial will exclusively use FREE nodes, Heroku still requires you to enter in a credit card in order to spin up Redis databases (which we'll need to do). You can do that at https://heroku.com/verify

### Step 3: Install Dependencies

You'll need to have the following dependencies installed:

- Heroku Toolbelt: https://toolbelt.heroku.com/
- Pip: https://www.makeuseof.com/tag/install-pip-for-python/
- Git: https://gist.github.com/derhuerst/1b15ff4652a867391f03
- PySyft: https://github.com/OpenMined/PySyft
- Grid: https://github.com/OpenMined/PyGrid

### Step 4: Login to Heroku

Run "heroku login" from the command line and follow the instructions.

### Step 5: Import Torch, PySyft, and Grid

Now we can import the dependencies we need.

In [1]:
import syft as sy
import torch as th
import grid as gr
from grid.deploy import HerokuNodeDeployment
hook = sy.TorchHook(th)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/allison/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [2]:
worker_id = "allisonsnode"

environment = {
    "ID": worker_id,
    "GRID_NETWORK_URL": "http://opengridnetwork.herokuapp.com",
    "ADDRESS": f"http://{worker_id}.herokuapp.com/"
}

node_component = HerokuNodeDeployment(worker_id, env_vars=environment, stack="container", dev_user="allibell", branch="fix_heroku_node_tutorial")

### Step 6: Launch and Connect to a Worker

Then you can run the code below - this will run various scripts to launch a node on heroku and then connect to it. If the script errors, just follow the instructions to correct the error and re-run it.

In [3]:
node_component.deploy()

Step 0: Checking Dependencies
	Checking for git dependency...DONE!
	Checking for heroku --version dependency...DONE!
	Checking for pip dependency...DONE!
	Checking to see if heroku is logged in...DONE!

Step 1: Making sure app name 'allisonsnode' is available
	['https://allisonsnode.herokuapp.com/ | https://git.heroku.com/allisonsnode.git\n']

Step 3: Cleaning up heroku/postgres checks...

Step 4: cleaning up git...
	
Step 5: cloning heroku app code from Github...
	
Step 6: copying app code from cloned repo...
	
Step 7: removing the rest of the cloned code...
	
Step 8: Initializing new github (for Heroku)...
	Initialized empty Git repository in /home/allison/PyGrid/examples/tmp/.git/
	
Step 9: Adding files to heroku github...
	
Step 10: Committing files to heroku github...
	[master (root-commit) b22a651] init
	 32 files changed, 2240 insertions(+)
	 create mode 100644 Dockerfile
	 create mode 100644 LICENSE
	 create mode 100644 Procfile
	 create mode 100644 README.md
	 create mode 1006

'https://allisonsnode.herokuapp.com'

In [4]:
worker = gr.WebsocketGridClient(hook, f"ws://{worker_id}.herokuapp.com/",id=worker_id)
worker.connect()

### Step 7: Use PySyft Like Normal

Now you can simply use the worker you created like you would any other normal PySyft worker. For more on how PySyft works, please see the PySyft tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials

In [5]:
x = th.tensor([1,2,3,4]).send(worker)
x

(Wrapper)>[PointerTensor | me:24991441299 -> allisonsnode:12685711744]

In [6]:
y = x + x
y

(Wrapper)>[PointerTensor | me:99783549090 -> allisonsnode:49422856673]

In [7]:
y.get()

tensor([2, 4, 6, 8])

### Step 8: Reconnecting to Same Node

If you run the same connect command that you did before, it will just connect to the existing node instead of launching it.

In [8]:
worker.close() # Disconnect from current connection

worker = gr.WebsocketGridClient(hook, address=node_component.deploy(), id=worker_id)
worker.connect()

Step 0: Checking Dependencies
	Checking for git dependency...DONE!
	Checking for heroku --version dependency...DONE!
	Checking for pip dependency...DONE!
	Checking to see if heroku is logged in...DONE!

Step 1: Making sure app name 'allisonsnode' is available
	[]
APP EXISTS: You can already connect to your app at https://allisonsnode.herokuapp.com


In [9]:
x = th.tensor([1,2,3,4]).send(worker)
x

(Wrapper)>[PointerTensor | me:24996282331 -> allisonsnode:65165153764]

#### Useful Tip

You can set check_deps=False and it'll run a little faster

In [10]:
worker.close() # Disconnect from current connection

node_component = HerokuNodeDeployment(worker_id, env_vars=environment, check_deps=False)
worker = gr.WebsocketGridClient(hook, address=node_component.deploy(), id=worker_id)
worker.connect()


Step 1: Making sure app name 'allisonsnode' is available
	[]
APP EXISTS: You can already connect to your app at https://allisonsnode.herokuapp.com
